### Running R in Jupyter notebook
To run R commands in a Jupyter notebook, you need to install the R kernel for Jupyter. From the terminal (not RStudio or R GUI), launch R (type R and press <return>), and run the following commands:
```r
install.packages('devtools')
devtools::install_github('IRkernel/IRkernel')
# If using R 3.5, run
IRkernel::installspec(name = 'ir35', displayname = 'R 3.5')
# otherwise, change 3.5 to the version of R that is installed on your computer
```

### IRanges
The `IRanges` package provides an efficient infrastructure for manipulating intervals/ranges of integers. This package is available on Bioconductor:  
https://bioconductor.org/packages/release/bioc/html/IRanges.html

To install this package, start R and enter:

```r
if (!requireNamespace("BiocManager", quietly = TRUE))  
    install.packages("BiocManager")  
BiocManager::install("IRanges", version = "3.8")
```

In [ ]:
# Load the IRanges package 
suppressMessages(library(IRanges))

`IRanges` are simple intervals that are defined by three parameters:
- start
- end
- width

To create an `IRanges` object, specify at least two of the above three parameters.

In [ ]:
# Construct IRanges by specifying the start and width of each interval
ir = IRanges(start=c( 1,  5,  8, 25),
             width=c(10, 11, 13,  6))
ir

In [ ]:
# Construct IRanges by specifying the start and the end of each interval
ir2 = IRanges(start=1:3, end=10)
ir2

In [ ]:
ir3 = IRanges(end=10, width=10:8)
ir3

The properties of the intervals can be accessed using the `start()`, `end()` and `width()` methods.

In [ ]:
# Example IRanges
ir

In [ ]:
# Get the start positions of all ranges
start(ir)

In [ ]:
# Get the end positions of all ranges
end(ir)

In [ ]:
# Get the widths of all ranges
width(ir)

In order to illustrate range operations, we’ll create a function to plot ranges.

In [ ]:
plotRanges = function(x, xlim=x, main=deparse(substitute(x)), col="gray", sep=0.5, ...){
  height = 1
  if (is(xlim, "IntegerRanges")) 
    xlim = c(min(start(xlim)), max(end(xlim)))
  bins = disjointBins(IRanges(start(x), end(x) + 1))
  plot.new()
  plot.window(xlim, c(0, max(bins)*(height + sep)))
  ybottom = bins * (sep + height) - height
  rect(start(x)-0.5, ybottom, end(x)+0.5, ybottom + height, col=col, ...)
  title(main)
  axis(1)
}
    
# Change plot size to 4in x 2.5in
library(repr)
options(repr.plot.width=4, repr.plot.height=2.5)

In [ ]:
# Print IRanges and plot the corresponding intervals
ir
plotRanges(ir)

### Operations with IRanges
There are many operations that can be applied to IRanges (see https://bioconductor.org/packages/release/bioc/vignettes/IRanges/inst/doc/IRangesOverview.pdf). The operations that are most useful for us are explained below.

In [ ]:
# Merging redundant ranges

# The function reduce() merges all IRanges into non-overlapping intervals
reduce(ir)

In [ ]:
# Plot the result of reduce()
options(repr.plot.width=4, repr.plot.height=3)
par(mfrow=c(2,1), mar=c(2, 4, 1, 1) + 0.1)
plotRanges(ir)
plotRanges(reduce(ir))

In [ ]:
# Counting overlapping ranges 
# (i.e. computing the coverage/occupancy of all ranges)

# The function coverage() counts the number of ranges over each position.
cov = coverage(ir)
cov

**Note:** The coverage is stored using the run-length encoding (Rle) format.  
One can convert this into an ordinary R vector using the `as.vector()` function.

In [ ]:
as.vector(cov)

In [ ]:
# Plot the result of coverage()
options(repr.plot.width=4, repr.plot.height=3)
par(mfrow=c(2,1), mar=c(2, 4, 1, 1) + 0.1)
plotRanges(ir)
cov = as.vector(cov)
plot(seq(1:length(cov)), cov, type='h', col="red", lwd=10, lend=1,
     xlab="Position", ylab="Coverage", bty="n")

In [ ]:
# Resizing

# Example IRanges
ir = IRanges(start=c(1, 5, 9), width=3)
ir

In [ ]:
# We can resize the ranges to have a new width, by keeping fix the start, 
# end, or center of the original intervals

# Resize to width 1, keeping the left end fixed
resize(ir, width=1, fix="start")

In [ ]:
# Resize to width 1, keeping the right end fixed
resize(ir, width=1, fix="end")

In [ ]:
# Resize to width 1, keeping the center fixed
resize(ir, width=1, fix="center")

In [ ]:
# Plot the results of the resize operations
options(repr.plot.width=4, repr.plot.height=3)
par(mfrow=c(4,1), mar=c(2, 4, 2, 1) + 0.1)
plotRanges(ir, xlim=c(0,12))
plotRanges(resize(ir, width=1, fix="start"), xlim=c(0,12))
plotRanges(resize(ir, width=1, fix="center"), xlim=c(0,12))
plotRanges(resize(ir, width=1, fix="end"), xlim=c(0,12))

More info on `IRanges`:
https://bioconductor.org/packages/release/bioc/vignettes/IRanges/inst/doc/IRangesOverview.pdf

### GRanges
The `GenomicRanges` package provides the `GRanges` class that allows manipulation of genomic ranges.  
https://bioconductor.org/packages/release/bioc/html/GenomicRanges.html

To install this package, run the following commands in R:

```r
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install("GenomicRanges", version = "3.8")
```

In [ ]:
# Load the GenomicRanges package
suppressMessages(library(GenomicRanges))

The `GRanges` class represents a collection of genomic ranges. Each genomic range is characterized by a chromosome name, a start and end location, and a strand. This class can be used to store the location of genomic features such as genes, transcripts, and binding sites. Genomic ranges can be created by using the `GRanges` constructor.

In [ ]:
# Construct a GRanges object
gr = GRanges(
    seqnames = c("chr1", "chr2", "chr3"),
    ranges = IRanges(start = c(1, 3, 5), width = 5),
    strand = c("+", "-", "+"))
gr

In [ ]:
# Occupancy of GRanges is computed using the coverage() method
cov = coverage(gr)
cov

In [ ]:
# Show coverage of chr3 as a normal vector
as.vector(cov$chr3)

In [ ]:
# GRanges can also be resized
# Resize to width 1 and keep the center fixed
gr
resize(gr, width=1, fix="center")

In [ ]:
# When we resize the GRanges and keep the "start" fixed 
# we observe something interesting
gr
resize(gr, width=1, fix="start")

When resizing `GRanges`, fix="start" indicates that the 5' end of the interval will remain fixed (not necessarily the left end).

In [ ]:
# Similarly, we can keep the 3' end fixed, by using the option fix="end"
gr
resize(gr, width=1, fix="end")

More info about `GRanges` can be found here: https://bioconductor.org/packages/release/bioc/vignettes/GenomicRanges/inst/doc/GenomicRangesIntroduction.html

Next we will use `GRanges` to illustrate the typical steps in a basic analysis of MNase-seq data.

### MNase-seq data analysis
For this we will use a few extra R packages. To install the necessary R packages, run the following commands in R:
```r
# To install the required Bioconductor packages, run:
if (!requireNamespace("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
BiocManager::install(
    c("genomation", "Gviz", "GenomicFeatures", "Rsamtools", "rtracklayer"))

# To install other packages used in this workshop, run:
install.packages("ggplot2")
install.packages("pryr")
install.packages("RMariaDB")
```

In [ ]:
# Load the necessary packages
suppressMessages({
    library(genomation)
    library(Gviz)
    library(GenomicFeatures)
    library(Rsamtools)
    library(rtracklayer)
    library(ggplot2)
    library(pryr)
})

First, we need to read the aligned reads from a `bam` file.

In [ ]:
# List all bam files from our bam folder
all_bam_files = list.files("bam/", pattern = ".bam$", full.names = TRUE)
all_bam_files

In [ ]:
# Load one of the bam files
bam_file = Rsamtools::BamFile('bam/MNase_400U_SRR3649299.bam')

In [ ]:
# Fields that are stored and can be imported from a bam file: 
scanBamWhat()

For now we will only need the following info:
   - qname - read (query) name
   - rname - reference (chromosome) name 
   - strand
   - pos   - start position of the read
   - isize - full length of the paired-end read

In [ ]:
# Import selected fields from the bam file
fields_to_load = c("qname", "rname", "strand", "pos", "isize")
param = ScanBamParam(what = fields_to_load)
aln = scanBam(bam_file, param = param)

In [ ]:
# Check the structure of the aln object
str(aln)

In [ ]:
# Let's subset the first element (all the information is in there)
aln = aln[[1]]

In [ ]:
# Get the total number of reads
length(aln$qname)

There are about 42 million reads, corresponding to about 21 million pairs (paired-end reads).

In [ ]:
# Construct GRanges from the paired-end reads
read_on_Watson_strand = (aln$strand == '+') & (aln$isize > 0)

reads = GRanges(seqnames = Rle(aln$rname[read_on_Watson_strand]),
                ranges = IRanges(start = aln$pos[read_on_Watson_strand],
                                 width = aln$isize[read_on_Watson_strand]))

reads

In [ ]:
# Check the size of memory that are occupied by aln and reads objects
pryr::object_size(aln)
pryr::object_size(reads)

The `aln` object occupies 2.7 GB of memory, while the `GRanges` occupy only 168 MB. To free some memory, the `aln` object can be removed now.

In [ ]:
# Remove aln from memory
rm(aln)

In [ ]:
# Check the number of paired-end reads that were mapped to each chromosome
no_of_reads = table(seqnames(reads))
no_of_reads

In [ ]:
# Get the chromosome lengths
chr_lengths = seqlengths(seqinfo(bam_file))
chr_lengths

In [ ]:
# Compute the sequencing depth (number of reads / chromosome)
no_of_reads_per_bp = no_of_reads / chr_lengths
no_of_reads_per_bp

There's a slight variability in the sequencing depth per chromosome, and this is why I prefer to normalize the coverage profiles per chromosome (more about this later).  

Chromosome 12 (chrXII) has a much higher sequencing depth though, because of the rDNA region, which we will mask later.

The mitochondrial DNA (chrM) has a much lower sequencing depth, but we don't care about chrM, which we will discard from the analysis anyway. 


In [ ]:
# The name of all chromosomes can be obtained using seqlevels()
seqlevels(reads)

In [ ]:
# Remove chrM
reads = dropSeqlevels(reads, "chrM", pruning.mode="coarse")
seqlevels(reads)

In [ ]:
# Check the problem with chrXII

# Compute the coverage on chrXII
cov = coverage(reads)$chrXII

plot(cov, type='l')

In [ ]:
# Get a few quantiles for the coverage of chrXII
quantile(cov, probs = c(0.95, 0.96, 0.97, 0.98, 0.99))

In [ ]:
# Let's see which regions have a coverage higher than 2000
slice(cov, lower=2000, rangesOnly=TRUE)

In [ ]:
# Create a GRanges object that contains all the problematic regions
# Discard the reads from the rDNA region
bad_regions = GRanges(seqnames = "chrXII",
                      ranges = IRanges(start = c(451000, 489000),
                                       end   = c(469000, 491000)))
bad_read_ind = overlapsAny(reads, bad_regions, ignore.strand=TRUE)
reads = reads[!bad_read_ind]

In [ ]:
# Check the number of paired-end reads that were mapped to each chromosome,
# after removing the rDNA reads
no_of_reads = table(seqnames(reads))
chr_lengths = chr_lengths[seqlevels(reads)]
no_of_reads_per_bp = no_of_reads / chr_lengths
no_of_reads_per_bp

After removing the mitochondrial DNA and the problematic regions of `chrXII`, the sequencing depth became much more even (between 1.53 and 1.65).

In [ ]:
# Save the sequencing depth to a csv file
write.csv(data.frame(Chrom = names(no_of_reads_per_bp), 
                     Reads_per_bp = as.vector(no_of_reads_per_bp)),
          file = "Sequencing_depth.MNase_400U.csv", 
          row.names = FALSE)

Next, we'll check the fragment length distribution.

In [ ]:
# Get read lengths. This is done easily using the width accessor (method) of GRanges 
read_length = width(reads)

# Plot a histogram of read lengths
h = hist(read_length, breaks = seq(from = 0.5, to = 1000.5, by = 1), plot=TRUE)

This plot shows the number of reads that have a given length. Since different samples have different total numbers of reads, it is more convenient to plot the percentage of reads for each length. We'll do this using `ggplot` to create a nicer figure.

In [ ]:
options(repr.plot.width=3, repr.plot.height=2)
df = data.frame(frag_length = h$mids, percentage_of_reads = 100*h$density)
p = ggplot(df, aes(x = frag_length, y = percentage_of_reads)) + 
  geom_line(colour="#56B4E9") +
  scale_x_continuous(limits = c(0, 550), expand = c(0, 0), 
                     breaks = seq(100, 500, 100)) +
  scale_y_continuous(expand = c(0, 0)) +
  theme_classic() +
  xlab("Fragment length (bp)") + 
  ylab("Percentage (%)")
p

In [ ]:
# Save the histogram to a pdf file
ggsave(filename = "Length_histogram.MNase_400U.pdf", 
       plot = p, width = 4, height = 3, units = "in")

In [ ]:
# Save the histogram to a csv file
write.csv(data.frame(Length = h$mids, Percentage = 100*h$density), 
          file = "Length_histogram.MNase_400U.csv", 
          row.names = FALSE)

Next, we will select the nucleosomal reads, with the length close to 150 bp (e.g. reads with the length between 120 bp and 180 bp), and we will analyze the distribution of the corresponding nucleosomes.

In [ ]:
# Size selection
Lmin = 120
Lmax = 180
size_filter = ((read_length >= Lmin) & (read_length <= Lmax))
reads.120_180 = reads[size_filter]

In [ ]:
# Compute the raw coverage of these reads
raw_occ = coverage(reads.120_180)

# Compute the average occupancy for each chromosome
chr_label = seqlevels(raw_occ)
for(chr in chr_label){
    cat(sprintf("Avg. occ. for %s is %0.3f\n", chr, mean(raw_occ[[chr]])))
}

In [ ]:
# Get the normalization factor, 1 / avg(occ), for each chromosome
occ_norm_factor = list()
for(chr in chr_label){
    occ_norm_factor[[chr]] = 1/mean(raw_occ[[chr]])
}

In [ ]:
# Compute the normalized occupancy (relative to chromosome average)
norm_occ = coverage(reads.120_180, weight = occ_norm_factor)

In [ ]:
# Check the averages of the normalized occupancy for each chromosome
for(chr in chr_label){
    cat(sprintf("Avg. occ. for %s is %0.3f\n", chr, mean(norm_occ[[chr]])))
}

Similarly, we can compute the genome-wide distribution of the nucleosome dyads (centers of DNA fragments).

In [ ]:
# Get the centers of all reads
dyad_pos.120_180 = resize(reads.120_180, width=1, fix="center")
head(dyad_pos.120_180)

In [ ]:
# Stack all dyad positions to construct the raw distribution of dyads
raw_dyads = coverage(dyad_pos.120_180)

In [ ]:
# Get the normalization factors for each chromosome
dyads_norm_factor = list()
for(chr in chr_label){
    dyads_norm_factor[[chr]] = 1/mean(raw_dyads[[chr]])
}

In [ ]:
# Compute the normalized dyad distribution (relative to chromosome average)
norm_dyads = coverage(dyad_pos.120_180, weight = dyads_norm_factor)

In [ ]:
# Check the averages of the normalized dyad distribution for each chromosome
for(chr in chr_label){
    cat(sprintf("Avg. dyad density for %s is %0.3f\n", chr, mean(norm_dyads[[chr]])))
}

In [ ]:
# Save profiles to BigWig files
rtracklayer::export.bw(norm_occ, "Norm_occ.MNase_400U.120-180.bw")
rtracklayer::export.bw(norm_dyads, "Norm_dyads.MNase_400U.120-180.bw")

### Visualizations
Next we will visualize the nucleosome distribution in different ways. The BigWig files that we created can be loaded in the IGV browser (http://software.broadinstitute.org/software/igv/), but we can also visualize the data directly in R.

First, let's look at a single genomic locus. Let's look at the _ARG1_ gene (`chrXV:219,211-220,473`).

In [ ]:
# Specify the range to plot
chr = "chrXV"
view_from = 217000
view_to = 223000

dyadsTrack = DataTrack(
    range = "Norm_dyads.MNase_400U.120-180.bw",
    type = "l",
    genome = "sacCer3",
    name = "Dyads"
)

occTrack = DataTrack(
    range = "Norm_occ.MNase_400U.120-180.bw",
    type = "l",
    genome = "sacCer3",
    name = "Occupancy"
)

# Gene annotation track
txdb = makeTxDbFromUCSC(genome="sacCer3", tablename = "sgdGene")
genesTrack = GeneRegionTrack(txdb, name="Genes", showId=TRUE)

# Genomic coordinate track
coordTrack = GenomeAxisTrack()

options(repr.plot.width=7, repr.plot.height=4)
plotTracks(
    list(dyadsTrack, occTrack, genesTrack, coordTrack),
    chromosome = "chrXV", 
    from = view_from, to = view_to
)